In [1]:
import cvxpy as cp
import torch 
from cvxpylayers.torch import CvxpyLayer
from numpy.random import rand
import numpy as np
import cProfile
from memory_profiler import profile

In [2]:
# starting w/ a simple LP

D = 10  # variable dimension
M = 50  # no of equality constraints
N = 200; # no of inequality constraints

x̂ = rand(D) # solution

_c = rand(D) # objective coeffs

_A = rand(M, D) # equality part
_b = np.matmul(_A, x̂)

_G = rand(N, D) # inequality part
_h = np.matmul(_G, x̂) + rand(N);

In [3]:
x = cp.Variable(D)
c = cp.Parameter(D)
A = cp.Parameter((M, D))
b = cp.Parameter(M)
G = cp.Parameter((N, D))
h = cp.Parameter(N)
constraints = [A@x == b, G@x <= h]
objective = cp.Minimize(c @ x)
problem = cp.Problem(objective, constraints)
assert problem.is_dpp()

In [4]:
cvxpylayer = CvxpyLayer(problem, parameters=[c,A,b,G,h], variables=[x])
c_t = torch.Tensor(_c)
A_t = torch.Tensor(_A)
b_t = torch.Tensor(_b)
G_t = torch.Tensor(_G)
h_t = torch.Tensor(_h)

c_t.requires_grad=False
A_t.requires_grad=True
b_t.requires_grad=True
G_t.requires_grad=False
h_t.requires_grad=False

# solve the problem
solution, = cvxpylayer(c_t, A_t, b_t, G_t, h_t)

In [5]:
# sanity check - constraints must satisfy
x_ = torch.Tensor(solution)
assert b_t.allclose(A_t@x_)
assert torch.all(h_t.gt(G_t@x_))

In [6]:
# compute the gradient of the sum of the solution with respect to A, b
cProfile.run('solution.sum().backward()')

         2113 function calls (2099 primitive calls) in 0.022 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(all)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(allclose)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(append)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(atleast_2d)
       32    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(can_cast)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(concatenate)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(cumsum)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(empty_like)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(expand_dims)
        1    0.000    0.00

/home/pika/programFiles/anaconda3/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


In [7]:
@profile 
def myGrad(solution):
    solution.sum().backward()

In [8]:
myGrad(solution)

ERROR: Could not find file <ipython-input-7-0ea92fd07375>
NOTE: %mprun can only be used on functions defined in physical files, and not in the IPython environment.
